# RBPに積載できる荷量の推定
## 概要
* 課題：RBPに積載できるケース数の推定量を算出したい
* 今回は上記の課題を3次元ビンパッキング問題として解いた
* 使用したパッケージ：`py3dbp`
  * [こちら](https://github.com/enzoruiz/3dbinpacking/blob/master/erick_dube_507-034.pdf)の論文を実装したPythonのパッケージ
  * [Github](https://github.com/enzoruiz/3dbinpacking)

## 算出時の主なロジック
* 大きなケースから順に積み込む
* 空きスペースが小さいRBPから順に埋める
* ケースがRBPの空きスペースに収まるまでケースを回転させる
  * 回転の順番は固定
  * 回転させても収まらない場合は、新しいRBPに積み込む
  * 回転の順番は[元の論文](https://github.com/enzoruiz/3dbinpacking/blob/master/erick_dube_507-034.pdf)のFig1とTable1を参照
* ケースの収まりを確認する時は、横方向 -> 上方向 -> 奥方向
* 荷物の重さについては、`RBPの上限 > 積載した荷物の合計` となっているかの確認のみで、「軽いものの上に重たいものが乗らないように」といった条件は含まれない

## その他
### 積載ロジックについて
* 回転の順番、種類は変更が可能（ex. 水平方向には回転させるけど、上下は逆さまにしない）
* 重さの条件については、積み込む順番を `重いもの順にソート -> 大きい順にソート`にすることで、重いケースが下に来るようにすることができそう

### 出力について
* 各ケースの位置情報も出力されるため、大変そうだけど頑張れば可視化も可能
* PowerBIでPythonのスクリプトを実行する or Azureの別のサービスで計算をして出力値を連携するなど、細見さんの使いやすいようにデータを提供することができそう

In [ ]:
import pandas as pd
from py3dbp import Packer, Bin, Item

今回は[クロネコボックスのサイズ](https://market.kuronekoyamato.co.jp/market/MarketTopAction_doViewGoodsDetail.action?struts.token.name=token&token=IL0TH5RK5CWLKW7ZHQJDZWNN0M3TG59D&ggId=1+++++++++&cmid=6b9214d38a484033)を想定した

|変数名|クロネコボックス|サイズ|
|---|---|---|
|b1|クロネコボックス6|20cm x 27cm x 13cm|
|b2|クロネコボックス8|27cm x 38cm x 15cm|
|b3|クロネコボックス10|27cm x 38cm x 35cm|
|b4|クロネコボックス12|38cm x 53cm x 29cm|
|b5|クロネコボックス14|40cm x 60cm x 40cm|

以下で各ボックスの個数を指定

In [ ]:
# ここを適宜変更
n_b1 = 50
n_b2 = 50
n_b3 = 30
n_b4 = 20
n_b5 = 20

# RBPのマージン(単位はcm、幅と奥行きから差し引く)
m = 10

In [ ]:
def generate_result_df() -> pd.DataFrame:
  # check the result
  dict_bins = dict()
  box_names = ["b1","b2","b3","b4","b5"]

  for b in packer.bins:
      if b.items:
          s = pd.Series([b.items[i].name for i in range(len(b.items))])
          d = dict()
          for n in box_names:
              d[n] = s.str.contains(n).sum()
          dict_bins[b.name] = d

  return pd.DataFrame.from_dict(dict_bins, orient="index")

In [ ]:
packer = Packer()

In [ ]:
'''
params: name, width, height, depth, weight
unit: length->mm, weight->kg
'''
#bin（今はRBPぴったりのサイズを上限にしており、はみ出しはNG）
for i in range(100):
    packer.add_bin(Bin(f'rbp{i}', 1040-m, 1040-m, 1700, 500))

# box1
for i in range(n_b1):
    packer.add_item(Item(f'b1_{i}', 200, 270, 130, 1))
# box2
for i in range(n_b2):
    packer.add_item(Item(f'b2_{i}', 270, 380, 150, 1))
# box3
for i in range(n_b3):
    packer.add_item(Item(f'b3_{i}', 270, 380, 350, 1))
# box4
for i in range(n_b4):
    packer.add_item(Item(f'b4_{i}', 380, 530, 290, 1))
# box5
for i in range(n_b5):
    packer.add_item(Item(f'b5_{i}', 400, 600, 400, 1))

各RBPに積載されたボックスの数を表示

In [ ]:
%%time
packer.pack(bigger_first=True, distribute_items=True)
df_res_bg = generate_result_df()
display(df_res_bg)

,b1,b2,b3,b4,b5
rbp0,9,6,1,0,14
rbp1,11,6,2,12,6
rbp2,27,8,18,8,0
rbp3,3,30,9,0,0


CPU times: user 2.02 s, sys: 13 ms, total: 2.03 s
Wall time: 2.18 s
